In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%ls
%cd CNNtention/

In [ ]:
import pipeline_template, mlflow, torch
from experiment_configs import ExperimentConfig, ModelConfig, DataConfig
from models.resnet_cbam import ResnetCBAM
from models.original_basic_block import OriginalBasicBlock
from skorch.callbacks import LRScheduler
from torch.optim.lr_scheduler import MultiStepLR
from torchvision.transforms import transforms
mlflow.set_tracking_uri('file:./mlruns')

In [ ]:
config={
    "cbam_att_final": lambda: ExperimentConfig(
        "CBAM Attention Final",
        DataConfig(name='CIFAR-10',
                   test_size=10000,
                   train_transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(0.5),
                       transforms.RandomCrop(32, padding=4),
                       transforms.ToTensor(),
                       transforms.Normalize(mean=[0.4918687901200927, 0.49185976472299225, 0.4918583862227116], std=[0.24697121702736, 0.24696766978537033, 0.2469719877121087])
                   ]),
                   test_transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean=[0.4918687901200927, 0.49185976472299225, 0.4918583862227116], std=[0.24697121702736, 0.24696766978537033, 0.2469719877121087])
                   ])),
        ModelConfig(model=ResnetCBAM(OriginalBasicBlock, [3,3,3], 10, 16),
                    lr=0.005,
                    optimizer=torch.optim.Adam,
                    weight_decay=0.0001,
                    use_reg=True,
                    batch_size=128,
                    max_epochs=1,
                    momentum=0.9,
                    train_split=None,
                    scheduler=LRScheduler(
                        policy=MultiStepLR,
                        milestones=[
                            16000 // (45000 // 128), # == 45
                            32000 // (45000 // 128), # == 91
                            48000 // (45000 // 128) # == 136
                        ],
                        gamma=0.1 # this is the multiplication factor ("divide it by 10")
                    ),
                    log_model=True,
                    use_early_stopping=True,
                    early_stopping_params={
                        'patience': 20,
                        'threshold': 0.001,
                        'threshold_mode': 'rel'
                    },
                    add_test_set_eval=True,
                    )
    )}

In [ ]:
pipeline_template.main('cbam_att_final', True, config)